In [ ]:
# import the necessary packages
import tensorflow.keras
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Input, Add
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

In [ ]:
class ResidualNet:
    @staticmethod
    def build(width, height, depth, classes, finalact):
        X_Input = Input(shape=(height, width, depth))

        # Common Block
        X0 = Conv2D(16, (3, 3), padding="same", strides=(1, 1), name="con_layer1")(X_Input)
        X0 = BatchNormalization(axis=3, name="B1")(X0)
        X0 = Activation("relu")(X0)
        X0 = MaxPooling2D(pool_size=(2, 2), strides=(1, 1), name="max1")(X0)
        X0 = Activation("relu")(X0)

        # Block 1
        X = Conv2D(32, (3, 3), padding="same", strides=(1, 1), name="con_layer2")(X0)
        X = BatchNormalization(axis=3, name="B2")(X)
        X = Activation("relu")(X)
        X = MaxPooling2D(pool_size=(1, 1), strides=(1, 1))(X)
        X = Activation("relu")(X)

        # Block 2
        X = Conv2D(32, (5, 5), padding="same", strides=(1, 1), name="con_layer3")(X)
        X = Activation("relu")(X)
        X = BatchNormalization(axis=3, name="B3")(X)
        # X = MaxPooling2D(pool_size=(1, 1), strides=(1, 1))(X)
        X = Activation("relu")(X)

        # Block 3
        X = Conv2D(64, (3, 3), padding="same", strides=(2, 2), name="con_layer4")(X)
        X = BatchNormalization(axis=3, name="B4")(X)
        X = Activation("relu")(X)
        # X = MaxPooling2D(pool_size=(1, 1), strides=(1, 1))(X)

        # Skipping Block 1
        XS = Conv2D(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer=glorot_uniform(seed=1),
                    name="con_layer5")(X0)
        XS = BatchNormalization(axis=3, name="B5")(XS)
        XS = Activation("relu")(XS)
########################################################################################################################

        # Adding layers
        X = tensorflow.keras.layers.Add()([X, XS])
        X = Activation("relu")(X)


        # Block 4
        X1 = Conv2D(64, (3, 3), padding="same", strides=(1, 1), name="con_layer7")(X)
        X1 = BatchNormalization(axis=3, name="B6")(X1)
        X1 = Activation("relu")(X1)
        X1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X1)
        X1 = Activation("relu")(X1)


        # Block 5
        X = Conv2D(128, (3, 3), padding="same", strides=(1, 1), name="con_layer8")(X1)
        X = BatchNormalization(axis=3, name="B7")(X)
        X = Activation("relu")(X)
        # X = MaxPooling2D(pool_size=(1, 1), strides=(1, 1))(X)

        # Block 6
        X = Conv2D(128, (3, 3), padding="same", strides=(2, 2), name="con_layer9")(X) # Strides 1
        X = BatchNormalization(axis=3, name="B8")(X)
        X = Activation("relu")(X)
        # X = MaxPooling2D(pool_size=(1, 1), strides=(2, 2))(X)

        # Skipping Block 2
        XS = Conv2D(128, (3, 3), strides=(2, 2), padding='same', kernel_initializer=glorot_uniform(seed=1),
                    name="con_layer10")(X1)
        XS = BatchNormalization(axis=3, name="B9")(XS)
        XS = Activation("relu")(XS)

########################################################################################################################

        # Adding layers
        X = tensorflow.keras.layers.Add()([X, XS])
        X = Activation("relu")(X)

        # Block 7
        X2 = Conv2D(256, (3, 3), padding="same", strides=(1, 1), name="con_layer11")(X)
        X2 = BatchNormalization(axis=3, name="B10")(X2)
        X2 = Activation("relu")(X2)
        X2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X2)
        X2 = Activation("relu")(X2)

        # Block 8
        X = Conv2D(256, (3, 3), padding="same", strides=(1, 1), name="con_layer12")(X2)
        X = BatchNormalization(axis=3, name="B11")(X)
        X = Activation("relu")(X)
        # X = MaxPooling2D(pool_size=(1, 1), strides=(1, 1))(X)

        # Block 9
        X = Conv2D(512, (3, 3), padding="same", strides=(2, 2), name="con_layer13")(X)  # strides 1
        X = BatchNormalization(axis=3, name="B12")(X)
        X = Activation("relu")(X)
        # X = MaxPooling2D(pool_size=(1, 1), strides=(2, 2))(X)

        # Skipping Block 3
        XS = Conv2D(512, (3, 3), strides=(2, 2), padding='same', kernel_initializer=glorot_uniform(seed=1),
                    name="con_layer14")(X2)
        XS = BatchNormalization(axis=3, name="B13")(XS)
        XS = Activation("relu")(XS)
        print(X)
        print(XS)
########################################################################################################################

        # Adding layers
        X = tensorflow.keras.layers.Add()([X, XS])
        X = Activation("relu")(X)

        X = Conv2D(256, (3, 3), padding="same", strides=(1, 1), name="con_layer15")(X)
        X = MaxPooling2D(pool_size=(1, 1), strides=(1, 1))(X)
        print(X.shape)
        print(XS.shape)

        X = Flatten()(X)
        X = Dense(128)(X)
        X = Dropout(0.5)(X)
        output = Dense(classes, activation=finalact)(X)

        model = Model(inputs=[X_Input], outputs=output)

        print(model.summary())
        return model